In [1]:
import random
import os

In [2]:
def extended_euclid(a, b):
    if b == 0:
        return a, 1, 0
    gcd, x1, y1 = extended_euclid(b, a % b)
    x = y1
    y = x1 - (a // b) * y1
    return gcd, x, y

def modulo_inverse(a,m):
    if extended_euclid(a,m)[0]==1:
        return extended_euclid(a,m)[1]%m
    else:
        return 0

## генерація чисел (1,2 пункт)

In [3]:
def horner_scheme(x,a,n):
    a_bin=str(bin(a))[2:]
    x_pow=1
    for i in a_bin:
        temp_x=x**int(i)
        x_pow=((x_pow**2)*temp_x)%n
    return x_pow

def miller_rabin_test(p:int,k:int):
    '''
    p: число, яке перевіряється на простоту
    k: лічильник
    '''
    s=0
    d=int(p)-1
    while d%2==0:
        s=s+1
        d=d//2
    for _ in range(k):
        x=random.randint(2,p-1)
        if extended_euclid(x,p)[0]>1:
            return False
        if horner_scheme(x,d,p)==1 or horner_scheme(x,d,p)==p-1:
            continue
        else:
            for r in range(0,s):
                power=d*(2**r)
                xr=horner_scheme(x,power,p)
                if xr==p-1:
                    break
                elif xr==1:
                    return False
                if r==s-1:
                    return False
    return True
    

def generate_prime_numbers():
    prime_numbers=[]
    f=open('test_numbers.txt','a')
    f.truncate(0)
    while True:
        p=int(random.getrandbits(256))
        if p % 2 == 0:
            p+=1
        result=miller_rabin_test(p,3)
        if result:
            prime_numbers.append(p)
        else:
            f.write(f'{str(p)}\n')
        if len(prime_numbers)==4: 
            break
    f.close()
    #pq<p1q1
    prime_numbers.sort()
    return prime_numbers
nums=generate_prime_numbers()
p=nums[0]
q=nums[1]
p1=nums[2]
q1=nums[3]
print(f'''
p={p}
q={q}

p1={p1}
q1={q1}''')



p=18237052018850384751518250401777005005759020965879163026290152325594413407319
q=32526987514378734599428393420423246762281375026882375641346930385020386435781

p1=91549199738910757513108051517500236699824382547295379487907369768880431742131
q1=96802940045247444637964911834549636749334213569791865162906428700673007037877


## 3 пункт

In [4]:
def generate_RSA_keys(p,q):
    n=p*q
    phi=(p-1)*(q-1)
    e=2**16 + 1
    d=modulo_inverse(e,phi)
    return [(e,n),(d,p,q)]

A_keys=generate_RSA_keys(p,q)
B_keys=generate_RSA_keys(p1,q1)
print(f'''
A public key: 
\t e={A_keys[0][0]}
\t n={A_keys[0][1]}
A private key: 
\t d={A_keys[1][0]}
\t p={A_keys[1][1]}
\t q={A_keys[1][2]}

B public key: 
\t e1={B_keys[0][0]}
\t n1={B_keys[0][1]}
B private key:
\t d1={B_keys[1][0]}
\t p1={B_keys[1][1]}
\t q1={B_keys[1][2]}
''')


        



A public key: 
	 e=65537
	 n=593196363316221960037844197364380098454291968720920100574265510082183786638594156036281446177371749260689467107572079268864705718787780799395674188881139
A private key: 
	 d=257754441584998592489703361587278210227548902044122277553952102333191139235940006010214061936940329830790685540270864015483991553126956263180194946696313
	 p=18237052018850384751518250401777005005759020965879163026290152325594413407319
	 q=32526987514378734599428393420423246762281375026882375641346930385020386435781

B public key: 
	 e1=65537
	 n1=8862231693516161071732304186095196230818879422419460644994004710956744882054056612097576667639739757312261582851432196922602260809192895021060471013695887
B private key:
	 d1=5229956711146849041157187492583087560052351809534996412801747504477822681207797256849223844469661106792780034140355402144058174356199239107144457758631713
	 p1=91549199738910757513108051517500236699824382547295379487907369768880431742131
	 q1=9680294004524744463796491183

# 4 пункт

In [5]:
def encrypt_RSA_message(M,public_keys):
    e,n=public_keys
    C=horner_scheme(M,e,n)
    return C

def decrypt_RSA_message(C,private_keys):
    d=private_keys[0]
    p=private_keys[1]
    q=private_keys[2]
    M=horner_scheme(C,d,p*q)
    return M

def signature_c(m,private_keys):
    d=private_keys[0]
    p=private_keys[1]
    q=private_keys[2]
    s=horner_scheme(m,d,p*q)
    return s
    
def signature_verification(m,s,public_keys):
    n=public_keys[1]
    e=public_keys[0]
    return horner_scheme(s,e,n)==m

# M=3
# C=encrypt_RSA_message(M,B_keys[0])
# M1=decrypt_RSA_message(C,B_keys)
# print(M1)

In [6]:
M=random.getrandbits(256)
print(f'Original message: {M}')
A_C=encrypt_RSA_message(M,A_keys[0])
A_M=decrypt_RSA_message(A_C,A_keys[1])
A_S=signature_c(M,A_keys[1])
print(f'''
A encrypt= {A_C}
A decrypt= {A_M}
M=A_M {M==A_M}
A sign: {A_S}
Verification state: {signature_verification(M,A_S,A_keys[0])}''')



B_C=encrypt_RSA_message(M,B_keys[0])
B_M=decrypt_RSA_message(B_C,B_keys[1])
B_S=signature_c(M,B_keys[1])
print(f'''
B encrypt= {B_C}
B decrypt= {B_M}
M=B_M {M==B_M}
B sign: {B_S}
Verification state: {signature_verification(M,B_S,B_keys[0])}
''')


Original message: 104653148630024620908657777697818121088205647423862821678288211741772485209906

A encrypt= 323526342463364835041978156185973167256338125612464652092977947660391476808964798190424475096045187018067548821609715887858465188420806233302807955536031
A decrypt= 104653148630024620908657777697818121088205647423862821678288211741772485209906
M=A_M True
A sign: 155635832870056399151862382744995620809154482953504098284062103203898290443503291203077006344342316188641355874877594410446547430687945767511945941707611
Verification state: True

B encrypt= 3470375061654699735762034936654611171173904327660635103541927569059596884422445801756522080582829151757001760332179582793547958359110150339048908458427076
B decrypt= 104653148630024620908657777697818121088205647423862821678288211741772485209906
M=B_M True
B sign: 271237968587290050481214359286987174751801209077304428388878462127561286063229738791685812261663601453362749195976710846046590237603577786894470299977202
Verification state:

# 5 пункт

In [7]:
def key_send(k,A_keys,B_public_keys):
    e=A_keys[0][0]
    n=A_keys[0][1]
    d=A_keys[1][0]
    e1=B_public_keys[0]
    n1=B_public_keys[1]

    k1=horner_scheme(k,e1,n1)
    s=horner_scheme(k,d,n)
    s1=horner_scheme(s,e1,n1)
    return (k1,s1)

def key_receive(key,B_keys,A_public_keys):
    k1=key[0]
    s1=key[1]
    n1=B_keys[0][1]
    d1=B_keys[1][0]

    s=horner_scheme(s1,d1,n1)
    return (k,s)

def key_authentication(key,A_public_keys):
    k=key[0]
    s=key[1]
    e=A_public_keys[0]
    n=A_public_keys[1]
    temp_k=horner_scheme(s,e,n)
    return temp_k==k 



In [8]:
k=random.randint(0,A_keys[0][1])
print(f'A generate secret k \n{k}')
A_sk=key_send(k,A_keys,B_keys[0])
print(f'''
A formed message (k1,s1):
k1={A_sk[0]}
s1={A_sk[1]}
''')
B_rk=key_receive(A_sk,B_keys,A_keys[0])
print(f''' B received (k1,s1) and formed message (k,s):
k={B_rk[0]}
s={B_rk[1]}
''')
verification_state=key_authentication(B_rk,A_keys[0])
print(f'Verification state: {verification_state}')

A generate secret k 
444351368892816475464708766805631952383840389914696807835605777572347395248115332252281472220746959773866349839871256792930486690808033120721354744097094

A formed message (k1,s1):
k1=3920073274341323080792740515701606690813183015108501164513758380843864914468764165212222167032731483273510918477629105712485956035465121780254156969134574
s1=8726855171203952123165828936619256741894851047291441733004294835369092310669990558480003660263421525030309822443410800955295318527662865910763749766580852

 B received (k1,s1) and formed message (k,s):
k=444351368892816475464708766805631952383840389914696807835605777572347395248115332252281472220746959773866349839871256792930486690808033120721354744097094
s=469356815237915276940631797307256289319767208592680823119211715081138014347334253906550460374234242041455619039064850957666802646807589828581709544100185

Verification state: True


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=29a10a5d-a94c-42c7-bb8b-5a64cb0a6a32' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>